In [14]:
import pandas as pd
import requests
import csv
from io import StringIO
import chardet

In [15]:
response = requests.get('https://opendata-ajuntament.barcelona.cat/data/es/dataset/est-mercat-immobiliari-compravenda-sup/resource/56b5a20f-078f-40f3-a637-bfa455d0ce1d/download')

In [16]:
if response.status_code == 200:
    # Detecta automáticamente la codificación del contenido
    encoding = chardet.detect(response.content)['encoding']
    
    # Decodifica el contenido con la codificación detectada
    contenido = response.content.decode(encoding)
    
    # Utilizamos StringIO para convertir el contenido en un archivo que pd.read_csv pueda leer
    csv_file = StringIO(contenido)
    
    # Utilizamos pd.read_csv para leer el contenido CSV
    df = pd.read_csv(csv_file)

else:
    print("La solicitud no fue exitosa. Código de estado:", response.status_code)


In [17]:
df.to_parquet('data_lake/compravenda_sup.parquet')

In [18]:
response = requests.get('https://opendata-ajuntament.barcelona.cat/data/dataset/c9b9b5e5-b02f-4b47-892b-79acf0191802/resource/44916a85-462d-4c1c-b3a5-98d9e161c299/download')

In [19]:
if response.status_code == 200:
    # Detecta automáticamente la codificación del contenido
    encoding = chardet.detect(response.content)['encoding']
    
    # Decodifica el contenido con la codificación detectada
    contenido = response.content.decode(encoding)
    
    # Utilizamos StringIO para convertir el contenido en un archivo que pd.read_csv pueda leer
    csv_file = StringIO(contenido)
    
    # Utilizamos pd.read_csv para leer el contenido CSV
    df = pd.read_csv(csv_file)

else:
    print("La solicitud no fue exitosa. Código de estado:", response.status_code)

In [20]:
df

,Any,Codi_Districte,Nom_Districte,Codi_Barri,Nom_Barri,Seccio_Censal,Import_Euros
0,2021,1,Ciutat Vella,1,el Raval,1,28484
1,2021,1,Ciutat Vella,1,el Raval,2,23611
2,2021,1,Ciutat Vella,1,el Raval,3,26607
3,2021,1,Ciutat Vella,1,el Raval,4,28688
4,2021,1,Ciutat Vella,1,el Raval,5,23769
...,...,...,...,...,...,...,...
1063,2021,10,Sant Martí,73,la Verneda i la Pau,143,34553
1064,2021,10,Sant Martí,65,el Clot,234,38965
1065,2021,10,Sant Martí,69,Diagonal Mar i el Front Marítim del Poblenou,235,29774
1066,2021,10,Sant Martí,69,Diagonal Mar i el Front Marítim del Poblenou,236,32925


In [21]:
df.to_parquet('data_lake/renda.parquet')

In [22]:
response = requests.get('https://www.kaggle.com/datasets/marshuu/rent-price-in-barcelona-2014-2022/download/datasetVersionNumber=1')

In [23]:
!kaggle datasets download -d marshuu/rent-price-in-barcelona-2014-2022

  0%|                                               | 0.00/33.6k [00:00<?, ?B/s]
100%|███████████████████████████████████████| 33.6k/33.6k [00:00<00:00, 453kB/s]


In [24]:
!unzip rent-price-in-barcelona-2014-2022.zip

Archive:  rent-price-in-barcelona-2014-2022.zip
  inflating: Barcelona_rent_price.csv  


In [25]:
pd.read_csv('Barcelona_rent_price.csv').to_parquet('data_lake/rent_price.parquet')

In [26]:
!rm rent-price-in-barcelona-2014-2022.zip Barcelona_rent_price.csv